In [82]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType
import talib
import requests
import time


symbol = "BTC-USDT"
timeframe = "1day"
market_type = "spot"
start_time = "2021-01-01 00:00:00"
end_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
# end_time = "2021-06-10 00:00:00"

skip_fetch = False


In [83]:

if not skip_fetch:
    print("Fetching data...")
    def fetch_kucoin_candles_chunk(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        time.sleep(0.2)  # Rate limiting
        base_url = "https://api.kucoin.com" if market_type.lower() == "spot" else "https://api-futures.kucoin.com"
        url = base_url + "/api/v1/market/candles"
        params = {"type": timeframe, "symbol": symbol.upper()}
        if start_time:
            params["startAt"] = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        if end_time:
            params["endAt"] = int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))
        try:
            resp = requests.get(url, params=params, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            if data.get("code") == "200000":
                return data["data"]
            else:
                raise Exception(f"KuCoin API error: {data}")
        except requests.exceptions.RequestException as e:
            raise Exception(f"Request failed: {str(e)}")

    def fetch_all_kucoin_candles(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        chunks = []
        current_end = end_time
        start_timestamp = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        
        while True:
            try:
                chunk = fetch_kucoin_candles_chunk(symbol, market_type, timeframe, start_time, current_end)
                if not chunk:
                    print("No more data available")
                    break
                earliest_ts = int(chunk[-1][0])
                print(f"Fetched {len(chunk)} candles from {datetime.fromtimestamp(earliest_ts)}")
                chunks.extend(chunk)
                if earliest_ts <= start_timestamp:
                    print("Reached start time")
                    break
                current_end = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(earliest_ts - 60))
            except Exception as e:
                print(f"Error occurred: {str(e)}")
                break
        
        if not chunks:
            return []
        
        chunks.sort(key=lambda x: x[0])  # Sort by timestamp
        result = [candle for candle in chunks if start_timestamp <= int(candle[0]) <= int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))]
        print(f"Total candles fetched: {len(result)}")
        return result


    data = fetch_all_kucoin_candles(symbol, market_type, timeframe, start_time, end_time)
    print(f"Final data length: {len(data)}")
    data
else:
    print("Skipping fetch...")

Fetching data...
Fetched 1458 candles from 2021-01-01 00:00:00
Reached start time
Total candles fetched: 1458
Final data length: 1458


In [84]:

# Assuming 'data' is the list of candlestick data
df = pd.DataFrame(data)
df[['timestamp', 'open', 'close', 'high', 'low']] = df[[0, 1, 2, 3, 4]]
df.drop([0, 1, 2, 3, 4, 5, 6], axis=1, inplace=True)

# Explicitly cast 'timestamp' to numeric type
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df[['open', 'close', 'high', 'low']] = df[['open', 'close', 'high', 'low']].astype(float)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1458 entries, 2021-01-01 00:00:00+00:00 to 2024-12-28 00:00:00+00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    1458 non-null   float64
 1   close   1458 non-null   float64
 2   high    1458 non-null   float64
 3   low     1458 non-null   float64
dtypes: float64(4)
memory usage: 57.0 KB


,open,close,high,low
timestamp,,,,
2021-01-01 00:00:00+00:00,28924.9,29333.2,29607.1,28633.0
2021-01-02 00:00:00+00:00,29333.2,32185.4,33286.8,28949.2
2021-01-03 00:00:00+00:00,32181.3,33000.4,34766.0,31500.0
2021-01-04 00:00:00+00:00,33005.9,31993.0,33600.2,27500.0
2021-01-05 00:00:00+00:00,31993.1,33938.6,34371.0,29806.1
...,...,...,...,...
2024-12-24 00:00:00+00:00,94885.9,98661.7,99476.5,93576.5
2024-12-25 00:00:00+00:00,98661.6,99434.8,99587.8,97622.7
2024-12-26 00:00:00+00:00,99434.9,95785.8,99970.8,95156.0


## creating indicator to set signals long ONLY for simplicity

In [85]:
import pandas as pd
import numpy as np
import vectorbt as vbt


skip_cell = False

if not skip_cell:
    print("This cell is executed.")


    def momentum_strategy(close, high, low, atr_length, ema_length,vola_multiplier, vol_window):
        # Calculate EMA
        EMA = vbt.IndicatorFactory.from_talib('EMA')
        ema = EMA.run(close, timeperiod=ema_length).real.to_numpy()

        # Calculate ATR
        ATR = vbt.IndicatorFactory.from_talib('ATR')
        atr = ATR.run(high, low, close, timeperiod=atr_length).real.to_numpy()

        def rolling_max(arr, window=vol_window):
            # Add proper handling for shape consistency
            result = np.full_like(arr, np.nan)
            for i in range(window-1, len(arr)):
                result[i] = np.max(arr[i-window+1:i+1])
            return result


        # def rolling_max_vectorized(arr, window):
        #     shape = (arr.size - window + 1, window)
        #     strides = (arr.strides[0], arr.strides[0])
        #     rolled = np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)
        #     max_vals = np.max(rolled, axis=1)
        #     result = np.full(arr.shape, np.nan)
        #     result[window-1:] = max_vals
        #     return result


        def compute_is_bearish_vol(high, low, atr, window=vol_window):
            # Calculate rolling max with proper shape handling
            rm = rolling_max(high, window=window)
            # Compare directly without padding
            return (rm - low) > (atr * vola_multiplier)

        # Calculate conditions
        is_bullish = close > ema
        is_bearish_vol = compute_is_bearish_vol(high, low, atr)
        
        # Ensure all arrays have the same shape
        is_caution = is_bullish & is_bearish_vol | (close < ema)
        signal_buy = is_bullish & ~is_caution

        conditions = [
            signal_buy,
            is_caution
        ]

        values = [10, 5]
        
        # Use np.select to create final signal
        signal = np.select(conditions, values, default=0)
        
        # Ensure signal has the same shape as input
        return signal.reshape(close.shape)

    # Create the indicator factory
    momentum_indicator = vbt.IndicatorFactory(
        class_name='MomentumStrategy',
        short_name='momentum',
        input_names=['Close', 'High', 'Low'],
        param_names=['atr_length', 'ema_length', 'vola_multiplier', 'vol_window'],
        output_names=['signal']
    ).from_apply_func(momentum_strategy)

    # Define parameter ranges
    atr_length = np.arange(4, 10, 1)  # Range from 5 to 14 with step 1
    ema_length = np.arange(100, 241, 20)  # Range from 10 to 30 with step 5
    vola_multiplier = [1.3,1.4,1.5,1.6,1.7,1.8]#np.arange(1, 2, 0.1)  # Range from 1 to 3 with step 0.1
    vol_window = np.arange(4, 8, 1)  # Range from 5 to 15 with step 1
    # 8	160	1.4	4 btc best setup 

    # Run the indicator
    signal = momentum_indicator.run(
        df['close'],df['high'],df['low'],
        atr_length=atr_length,
        ema_length=ema_length,
        vola_multiplier=vola_multiplier,
        vol_window=vol_window,
        param_product=True  
    )
    print(signal.signal.shape)
    df_indicator=signal.signal


else:
    print("This cell is skipped.")

This cell is executed.
(1458, 1152)


In [86]:

def rolling_max(arr, window):
    # Add proper handling for shape consistency
    result = np.full_like(arr, np.nan)
    for i in range(window-1, len(arr)):
        result[i] = np.max(arr[i-window+1:i+1])
    return result

def compute_is_bearish_vol(high, low, atr, vola_multiplier, window):
    # Calculate rolling max with proper shape handling
    rm = rolling_max(high, window=window)
    # Compare directly without padding
    print((rm - low) )
    return (rm - low) > (atr * vola_multiplier)

# Create an array with 20 integers
array = np.arange(1, 21)

# Set every 3rd number to be the highest
array[2::3] = np.max(array) + 1

# Create a low array with the same shape
low = np.full(array.shape, 1)

print("Array:", array)
print("Low:", low)

# Compute bearish volatility
result = compute_is_bearish_vol(array, low, 0.2, vola_multiplier=1.5, window=2)
print("Bearish Volatility:", result)

Array: [ 1  2 21  4  5 21  7  8 21 10 11 21 13 14 21 16 17 21 19 20]
Low: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[9223372036854775807                   1                  20
                  20                   4                  20
                  20                   7                  20
                  20                  10                  20
                  20                  13                  20
                  20                  16                  20
                  20                  19]
Bearish Volatility: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]


In [87]:
df_indicator

momentum_atr_length         4                                      ...   9  \
momentum_ema_length       100                                      ... 240   
momentum_vola_multiplier  1.3             1.4             1.5      ... 1.6   
momentum_vol_window         4   5   6   7   4   5   6   7   4   5  ...   6   
timestamp                                                          ...       
2021-01-01 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
2021-01-02 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
2021-01-03 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
2021-01-04 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
2021-01-05 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
...                        ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..   
2024-12-24 00:00:00+00:00  10  10   5   5  10  10   5   5  10  10  ...   5   
2024-12-25 00:00:00+00:00  10  10  10  10  10  10  10  10  10  10  ...  10   
2024-12-26 00:00:00+00:00  10  10  10  10  10  10  10  10  10  10  ...  10   
2024-12-27 00:00:00+00:00   5   5   5   5   5   5   5   5   5   5  ...  10   
2024-12-28 00:00:00+00:00   5   5   5   5   5   5   5   5   5   5  ...  10   

momentum_atr_length                                            
momentum_ema_length                                            
momentum_vola_multiplier      1.7             1.8              
momentum_vol_window         7   4   5   6   7   4   5   6   7  
timestamp                                                      
2021-01-01 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
2021-01-02 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
2021-01-03 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
2021-01-04 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
2021-01-05 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
...                        ..  ..  ..  ..  ..  ..  ..  ..  ..  
2024-12-24 00:00:00+00:00   5  10  10   5   5  10  10   5   5  
2024-12-25 00:00:00+00:00  10  10  10  10  10  10  10  10  10  
2024-12-26 00:00:00+00:00  10  10  10  10  10  10  10  10  10  
2024-12-27 00:00:00+00:00  10  10  10  10  10  10  10  10  10  
2024-12-28 00:00:00+00:00  10  10  10  10  10  10  10  10  10  

[1458 rows x 1152 columns]

In [88]:
def repeat_series_horizontally(series, target_shape):
    data = series.values.reshape(-1, 1)  # Convert to column vector
    return np.repeat(data, target_shape[1], axis=1)

nedded_copies = len(df_indicator.columns)
print('needed columns: ',nedded_copies)
print('needed rows: ',len(df_indicator))
print('shape:', df_indicator.shape)
print()

# copying the close price to the shape of the indicator
print('original shape:\n\n', df['close'])
dublicate_close = repeat_series_horizontally(df['close'], df_indicator.shape)
df_dublicate_close  = pd.DataFrame(dublicate_close,df.index)
df_dublicate_close 



needed columns:  1152
needed rows:  1458
shape: (1458, 1152)

original shape:

 timestamp
2021-01-01 00:00:00+00:00    29333.2
2021-01-02 00:00:00+00:00    32185.4
2021-01-03 00:00:00+00:00    33000.4
2021-01-04 00:00:00+00:00    31993.0
2021-01-05 00:00:00+00:00    33938.6
                              ...   
2024-12-24 00:00:00+00:00    98661.7
2024-12-25 00:00:00+00:00    99434.8
2024-12-26 00:00:00+00:00    95785.8
2024-12-27 00:00:00+00:00    94286.1
2024-12-28 00:00:00+00:00    94681.5
Name: close, Length: 1458, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,...,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2
2021-01-02 00:00:00+00:00,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,...,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4
2021-01-03 00:00:00+00:00,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,...,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4
2021-01-04 00:00:00+00:00,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,...,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0
2021-01-05 00:00:00+00:00,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,...,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24 00:00:00+00:00,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,...,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7,98661.7
2024-12-25 00:00:00+00:00,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,...,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8,99434.8
2024-12-26 00:00:00+00:00,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,...,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8,95785.8


## compare the shape of the copied closing prices 

In [89]:
df_custom_indiator = df_indicator

if df_custom_indiator.shape == df_dublicate_close.shape:
    df_renamed_close = df_dublicate_close
    df_renamed_close.columns = df_custom_indiator.columns
    print("Columns have been renamed ")
else:
    print("The DataFrames do not have the same shape.")

# Display the resulting DataFrame
df_renamed_close.head()
df_renamed_close

Columns have been renamed 


momentum_atr_length              4                                      \
momentum_ema_length            100                                       
momentum_vola_multiplier       1.3                                 1.4   
momentum_vol_window              4        5        6        7        4   
timestamp                                                                
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6   
...                            ...      ...      ...      ...      ...   
2024-12-24 00:00:00+00:00  98661.7  98661.7  98661.7  98661.7  98661.7   
2024-12-25 00:00:00+00:00  99434.8  99434.8  99434.8  99434.8  99434.8   
2024-12-26 00:00:00+00:00  95785.8  95785.8  95785.8  95785.8  95785.8   
2024-12-27 00:00:00+00:00  94286.1  94286.1  94286.1  94286.1  94286.1   
2024-12-28 00:00:00+00:00  94681.5  94681.5  94681.5  94681.5  94681.5   

momentum_atr_length                                                     ...  \
momentum_ema_length                                                     ...   
momentum_vola_multiplier                                  1.5           ...   
momentum_vol_window              5        6        7        4        5  ...   
timestamp                                                               ...   
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2  ...   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4  ...   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4  ...   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0  ...   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6  ...   
...                            ...      ...      ...      ...      ...  ...   
2024-12-24 00:00:00+00:00  98661.7  98661.7  98661.7  98661.7  98661.7  ...   
2024-12-25 00:00:00+00:00  99434.8  99434.8  99434.8  99434.8  99434.8  ...   
2024-12-26 00:00:00+00:00  95785.8  95785.8  95785.8  95785.8  95785.8  ...   
2024-12-27 00:00:00+00:00  94286.1  94286.1  94286.1  94286.1  94286.1  ...   
2024-12-28 00:00:00+00:00  94681.5  94681.5  94681.5  94681.5  94681.5  ...   

momentum_atr_length              9                                      \
momentum_ema_length            240                                       
momentum_vola_multiplier       1.6               1.7                     
momentum_vol_window              6        7        4        5        6   
timestamp                                                                
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6   
...                            ...      ...      ...      ...      ...   
2024-12-24 00:00:00+00:00  98661.7  98661.7  98661.7  98661.7  98661.7   
2024-12-25 00:00:00+00:00  99434.8  99434.8  99434.8  99434.8  99434.8   
2024-12-26 00:00:00+00:00  95785.8  95785.8  95785.8  95785.8  95785.8   
2024-12-27 00:00:00+00:00  94286.1  94286.1  94286.1  94286.1  94286.1   
2024-12-28 00:00:00+00:00  94681.5  94681.5  94681.5  94681.5  94681.5   

momentum_atr_length                                                     
momentum_ema_length                                                     
momentum_vola_multiplier                1.8                             
momentum_vol_window              7        4        5        6        7  
timestamp                                                               


In [90]:
atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()
print(type(atr_values))
print(atr_values.shape)

<class 'numpy.ndarray'>
(1458,)


In [91]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,entry_price,atr_values):
    close_price = c.close[c.i, c.col]
    # print("INDEX", c.i)
    # print("COL", c.col)
    # print("high", high[c.i])
    # print('open', open_[c.i])
    # print("close", close_price)
    # print('low', low[c.i])
    # print('position size :', c.position_now)
    # print('cash:', c.cash_now)
    # print('entries:', entries[c.i, c.col])
    # print('sl_prices:', sl_prices[c.i])
    # print('tp_prices:', tp_prices[c.i])
    # print('entry_price:', entry_price[c.i])
    # print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        if c.close[c.i-1, c.col] <= sl_prices[c.i]:
        # if low[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=sl_prices[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # print('sl hit at index',c.i)
            # print('sl order', value)
            return value

        # sl update for long position

        if c.i >= 5:  # Ensure there are enough bars to look back
            if (entries[c.i, c.col] == 5):  
                update2 = low[c.i] - atr_values[c.i] * 0.2
                if update2 > sl_prices[c.i]:
                    # print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= update2       


    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i, c.col] == 10:
            entry_price[:] = close_price
            order = vbt.portfolio.nb.order_nb(
                size=1,  # Adjusted order size
                price=close_price,  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.001,  # No fees
                slippage=0.002,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )

            sl_prices[:] = low[c.i] - atr_values[c.i] 
            # print('order', order)
            return order

    
    return vbt.portfolio.enums.NoOrder


close = df_renamed_close
entries = df_custom_indiator.to_numpy()

open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()

atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()

# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array

# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,
    tp_prices,
    entry_price,
    atr_values,  # Pass the SL prices array
    init_cash=500  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable




Total Return: momentum_atr_length  momentum_ema_length  momentum_vola_multiplier  momentum_vol_window
4                    100                  1.3                       4                      2.367689
                                                                    5                      2.321658
                                                                    6                      1.656423
                                                                    7                      1.350228
                                          1.4                       4                      2.565797
                                                                                             ...   
9                    240                  1.7                       7                      0.969743
                                          1.8                       4                      1.177088
                                                                    5                      1.39614

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(4, 100, 1.3, 4)",2021-04-11 00:00:00+00:00,0.008307,60127.615200,0.499500,Buy
1,1,"(4, 100, 1.3, 4)",2021-04-18 00:00:00+00:00,0.008307,60323.940187,0.501131,Sell
2,2,"(4, 100, 1.3, 4)",2021-04-27 00:00:00+00:00,0.009073,55124.328600,0.500130,Buy
3,3,"(4, 100, 1.3, 4)",2021-05-13 00:00:00+00:00,0.009073,53601.345257,0.486313,Sell
4,4,"(4, 100, 1.3, 4)",2021-07-28 00:00:00+00:00,0.012105,40095.531000,0.485341,Buy
...,...,...,...,...,...,...,...
80853,80853,"(9, 240, 1.8, 7)",2024-10-07 00:00:00+00:00,0.012104,62347.646400,0.754627,Buy
80854,80854,"(9, 240, 1.8, 7)",2024-11-05 00:00:00+00:00,0.012104,68454.379754,0.828541,Sell
80855,80855,"(9, 240, 1.8, 7)",2024-11-07 00:00:00+00:00,0.010878,76017.130800,0.826885,Buy
80856,80856,"(9, 240, 1.8, 7)",2024-12-20 00:00:00+00:00,0.010878,98910.989444,1.075916,Sell


In [92]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(50)



total_return  \
momentum_atr_length momentum_ema_length momentum_vola_multiplier momentum_vol_window                 
4                   100                 1.8                      5                        3.185993   
9                   160                 1.4                      4                        3.150536   
                                                                 5                        3.128440   
8                   160                 1.4                      4                        3.089379   
7                   100                 1.7                      4                        3.080139   
6                   100                 1.7                      4                        3.080041   
7                   160                 1.4                      5                        3.018535   
8                   100                 1.4                      4                        3.018317   
                    160                 1.4                      5                        3.010032   
9                   100                 1.4                      4                        3.003232   
4                   100                 1.7                      5                        2.991054   
7                   160                 1.4                      4                        2.988350   
5                   100                 1.7                      6                        2.978981   
9                   220                 1.4                      5                        2.973756   
8                   160                 1.5                      5                        2.956885   
9                   200                 1.4                      5                        2.953607   
                    160                 1.6                      4                        2.949225   
                                        1.5                      5                        2.945048   
                    200                 1.4                      4                        2.944209   
5                   100                 1.7                      5                        2.942572   
6                   100                 1.7                      6                        2.941360   
9                   140                 1.4                      4                        2.937764   
                    100                 1.4                      5                        2.933727   
7                   100                 1.4                      4                        2.919044   
5                   160                 1.6                      5                        2.917463   
9                   140                 1.4                      5                        2.912622   
6                   160                 1.4                      4                        2.895416   
8                   200                 1.4                      4                        2.886092   
6                   160                 1.4                      5                        2.884262   
8                   140                 1.4                      4                        2.879742   
                    160                 1.6                      4                        2.853489   
7                   200                 1.4                      5                        2.848356   
4                   160                 1.8                      5                        2.845528   
8                   200                 1.4                      5                        2.840213   
7                   160                 1.7                      4                        2.834820   
6                   160                 1.7                      4                        2.834728   
7                   100                 1.4                      5                        2.829006   
8                   160                 1.5                      4                        2.821806   
4     

In [98]:
# Access the setup for a specific name, e.g., '(4, 100, 1.0, 4)'
setup_name = (4, 100, 1.8, 5)
specific_pf = pf[setup_name]
specific_pf.plot().show()



In [97]:
specific_pf2 = pf.select_one(setup_name)
specific_pf2.total_return()

3.1859932730512317